# **응급상황 자동 인식 및 응급실 연계 서비스**
# **단계3 : 응급상황 연계(추천)**

## **0.미션**

단계 3에서는, 응급상황의 음성을 인식해서 텍스트로 변환하고, 변환된 텍스트를 다시 요약 및 핵심키워드 도출 작업을 수행합니다.  
이를 위해 사전학습된 모델을 API로 연결하여 활용합니다.

### 미션4 : 응급실 추천
* 응급실 위치와 응급전화 발신자 위치 기반 추천
* 두 좌표간 직선거리(Haversine)
    * 1) 500여 곳 응급실에 대해서, 거리 기반 가까운 응급실 찾기
    * 2) 좌표 구간을 설정하여 대상 응급실 범위를 좁힌 후, 거리 기반 가까운 응급실 찾기


## **1.환경설정**

### (1) 경로 설정

구글 드라이브 연결

#### 1) 구글 드라이브 폴더 생성
* 새 폴더(project6_2)를 생성하고
* 제공 받은 파일을 업로드

#### 2) 구글 드라이브 연결

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/project6_2/'

### (2) 라이브러리

#### 1) 필요한 라이브러리 설치

* requirements.txt 파일의 [경로 복사]를 한 후,
* 아래 경로에 붙여 넣기

In [ ]:
# 경로 : /content/drive/MyDrive/project6_2/requirements.txt
# 경로가 다른 경우 아래 코드의 경로 부분을 수정하세요.

!pip install -r /content/drive/MyDrive/project6_2/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


#### 2) 라이브러리 로딩

In [ ]:
#필요한 라이브러리 설치 및 불러우기
import os
import pandas as pd
import numpy as np

from haversine import haversine
import requests
import json

# 더 필요한 라이브러리 추가 -------------




### (3) 데이터 로딩
* 단계1에서 수집한 응급실 정보를 불러와서 데이터프레임으로 저장합니다.

In [ ]:
haversine((37.35861845,127.1150359),(36.81557078,127.1288439),unit='km')

60.39664937393305

## **2. 응급실 추천**


### (1) 직선거리 계산
- haversine formula
    * Haversine은 두 지점 간의 거리를 구할 때 사용하는 수학 공식으로, 지구의 구형 구조를 고려하여 위도와 경도를 기반으로 직선 거리를 계산한다.
- 세부사항
    * 하버사인 함수를 활용


#### 1) 하버사인 함수 사용 연습
* 임의의 두 좌표간 거리 계산
    * 응급실 데이터프레임을 열어서
    * 응급실 두 곳의 좌표를 확인하고
    * 두 지점의 거리를 계산해 봅시다
* 사용법 : haversine((위도1, 경도1), (위도2, 경도2), unit='km')


In [ ]:
df=pd.read_csv('/content/drive/MyDrive/project6_2/응급실정보.csv')

In [ ]:
df.head()

,기관명,기관주소,응급의료기관분류명,전화번호1,전화번호3,위도,경도
0,(의)내경의료재단울산제일병원,울산광역시 남구 남산로354번길 26 (신정동),응급실운영신고기관,052-220-3300,052-220-3334,35.548238,129.307011
1,(의)서일의료재단기장병원,부산광역시 기장군 기장읍 대청로72번길 6,지역응급의료기관,051-723-0171,051-723-2119,35.236029,129.216492
2,(의)성세의료재단 뉴성민병원,"인천광역시 서구 칠천왕로33번길 17 (석남동, 신석로 70(석남1동, 성민병원))",지역응급의료기관,032-726-1000,032-726-1190,37.508994,126.669479
3,(의)영문의료재단다보스병원,"경기도 용인시 처인구 백옥대로1082번길 18, 다보스종합병원 (김량장동)",지역응급의료센터,031-8021-2114,031-8021-2130,37.234641,127.210499
4,(의)효심의료재단용인서울병원,경기도 용인시 처인구 고림로 81 (고림동),지역응급의료기관,031-337-0114,031-336-0119,37.240316,127.214491


#### 2) 가장 가까운 응급실 3곳 추천하기1
* 세부사항
    * 입력된 좌표와 전체 응급실과 거리를 계산한 후
    * 가장 가까운 거리의 응급실 3 곳을 반환합니다.
* 이를 함수로 생성하고 테스트 해 봅시다.

In [ ]:
my_location=(37.339073,127.966817)

def recommend_hospital(my_location,df):
  distances=[]
  for index in range(len(df)):
    distance=haversine((df.iloc[index]['위도'],df.iloc[index]['경도']),my_location)
    distances.append(distance)
  df['거리']=distances
  df=df.sort_values(by='거리',ascending=True).reset_index(drop=True)
  return df[:3]

In [ ]:
recommend_hospital(my_location,df)

,기관명,기관주소,응급의료기관분류명,전화번호1,전화번호3,위도,경도,거리
0,의료법인성지의료재단성지병원,강원특별자치도 원주시 원일로 22 (인동),지역응급의료기관,033-760-3101,033-760-3119,37.345299,127.954009,1.327125
1,강원특별자치도원주의료원,"강원특별자치도 원주시 서원대로 387 (개운동, 지방공사강원도원주의료원)",지역응급의료기관,033-761-6911,033-760-4701,37.332496,127.948962,1.739747
2,연세대학교원주세브란스기독병원,강원특별자치도 원주시 일산로 20 (일산동),권역응급의료센터,033-741-0114,033-741-1641,37.347955,127.945645,2.116233


#### 3) 가장 가까운 응급실 3곳 추천하기2
* 문제점 : 입력 받은 좌표와 응급실 전체와의 거리를 모두 계산하는 것은 비효율 적입니다.
* 해결 방안 : 그래서 입력 받은 좌표를 기준으로 일정 범위 내에 해당되는 응급실에 대해서 거리를 계산하고 추천하도록 기존 함수를 수정 합니다.
* hint :
    * 입력 받은 위도, 경도 값에 ± α 하여 일정 범위 구간을 정하고
    * 응급실 정보에서 해당 구간을 먼저 조회한 후
    * 거리 계산

In [ ]:
my_location=(37.339073,127.966817)

def recommend_hospital2(my_location,df,alpha):

  #내 위치의 경도,위도
  my_lat,my_lon=my_location
  #범위 지정
  lat_min,lat_max=my_lat-alpha,my_lat+alpha
  lon_min,lon_max=my_lon-alpha,my_lon+alpha

  #지정한 범위의 응급실 필터링
  df=df[(df['위도']>=lat_min)&(df['위도']<=lat_max)&
        (df['경도']>=lon_min)&(df['경도']<=lon_max)]

  #지정한 범위의 응급실과 나의 위치의 거리 계산
  distances=[]
  for index in range(len(df)):
    distance=haversine((df.iloc[index]['위도'],df.iloc[index]['경도']),my_location)
    distances.append(distance)

  #거리 저장 및 오름차순으로 sort
  df['거리']=distances
  df=df.sort_values(by='거리',ascending=True).reset_index(drop=True)

  return df[:3]


In [ ]:
recommend_hospital2(my_location,df,.5)

<ipython-input-22-495ac39c3d4c>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['거리']=distances


,기관명,기관주소,응급의료기관분류명,전화번호1,전화번호3,위도,경도,거리
0,의료법인성지의료재단성지병원,강원특별자치도 원주시 원일로 22 (인동),지역응급의료기관,033-760-3101,033-760-3119,37.345299,127.954009,1.327125
1,강원특별자치도원주의료원,"강원특별자치도 원주시 서원대로 387 (개운동, 지방공사강원도원주의료원)",지역응급의료기관,033-761-6911,033-760-4701,37.332496,127.948962,1.739747
2,연세대학교원주세브란스기독병원,강원특별자치도 원주시 일산로 20 (일산동),권역응급의료센터,033-741-0114,033-741-1641,37.347955,127.945645,2.116233


### (2) [조 과제]고도화 : naver 지도 api 사용

* 이 부분은 조별 과제로 수행하게 됩니다.(개인과제 아님!)

* 세부사항
    * 두 지점간, 최단 도로거리, 소요 시간을 계산하는 함수를 생성하시오.
    * 함수 내용
        * 입력 : 두 지점의 위도, 경도, 네이버클라우드id, 암호키
        * 출력 : 도로거리(km)
    
    * 네이버 Maps API 활용
        * 사용할 API : Direction 5
        * 가이드 : https://guide.ncloud-docs.com/docs/ko/maps-direction5-api
        * 가이드를 활용해서 url, header, params를 구성합니다.
        * params의 옵션은 'trafast' (실시간 빠른 길 옵션)을 선택하시오.

#### 1) maps 클라이언트ID, 키 로딩

In [ ]:
c_id='5rpillllni'
c_key='2fsukZsIpH48SFzlZ8v0GoJ7fXTkiMO3eHJcge12'

#### 2) 함수 생성

In [ ]:
def get_dist(start_lat, start_lng, dest_lat, dest_lng, c_id, c_key):

    url = "https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving"
    headers = {
        "X-NCP-APIGW-API-KEY-ID": c_id,
        "X-NCP-APIGW-API-KEY": c_key,
    }
    #받아올 param
    params = {
        "start": f"{start_lng},{start_lat}",  # 출발지 (경도, 위도)
        "goal": f"{dest_lng},{dest_lat}",    # 목적지 (경도, 위도)
        "option": "trafast"  # 실시간 빠른 길 옵션
    }

    # 요청하고, 답변 받아오기
    response =requests.get(url,headers=headers,params=params)

    # 성공적으로 받아왔다면
    if response.status_code == 200:
        data = response.json()  # JSON 데이터를 파싱
        if data["code"] == 0:
            # 거리
            dist = data['route']['trafast'][0]['summary']['distance'] / 1000  # km로 변환
            # 소요 시간
            time = data['route']['trafast'][0]['summary']['duration'] / 60000  # ms를 분으로 변환
            return round(dist,2), round(time,2)


* 테스트

In [ ]:
get_dist(37.339073,127.966817,37.345299,127.954009,c_id,c_key)

(2.21, 7.44)

#### 3) 응급실 추천
* recommend_hospital2 함수를 참조해서 recommend_hospital3 만들기
    * 거리 계산 부분을 get_dist 함수로 대체
    * 입력 부분 수정

In [ ]:
def recommend_hospital3(my_location, df, alpha, c_id, c_key):

    # 내 위치의 경도, 위도
    my_lat, my_lon = my_location
    # 범위 지정
    lat_min, lat_max = my_lat - alpha, my_lat + alpha
    lon_min, lon_max = my_lon - alpha, my_lon + alpha

    # 지정한 범위의 병원 필터링
    df = df[(df['위도'] >= lat_min) & (df['위도'] <= lat_max) &
            (df['경도'] >= lon_min) & (df['경도'] <= lon_max)]

    # 지정한 범위의 병원과 내 위치 간 도로 거리 계산
    distances = []
    times = []
    for index in range(len(df)):
        hospital_lat = df.iloc[index]['위도']
        hospital_lon = df.iloc[index]['경도']

        # `get_dist`로 도로 거리 및 소요 시간 계산
        distance, time = get_dist(my_lat, my_lon, hospital_lat, hospital_lon, c_id, c_key)
        distances.append(distance)
        times.append(time)

    # 계산한 도로 거리 및 소요 시간 추가
    df['거리(km)'] = distances
    df['소요시간(분)'] = times

    # 거리 기준으로 정렬
    df = df.sort_values(by='거리', ascending=True).reset_index(drop=True)

    # 가장 가까운 병원 3곳 반환
    return df[:3]

In [ ]:
my_location=(37.339073,127.966817)

In [ ]:
recommend_hospital3(my_location, df, 0.5, c_id, c_key)

<ipython-input-39-6987db48c037>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['거리(km)'] = distances
<ipython-input-39-6987db48c037>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['소요시간(분)'] = times


,기관명,기관주소,응급의료기관분류명,전화번호1,전화번호3,위도,경도,거리,거리(km),소요시간(분)
0,의료법인성지의료재단성지병원,강원특별자치도 원주시 원일로 22 (인동),지역응급의료기관,033-760-3101,033-760-3119,37.345299,127.954009,1.327125,2.21,7.21
1,강원특별자치도원주의료원,"강원특별자치도 원주시 서원대로 387 (개운동, 지방공사강원도원주의료원)",지역응급의료기관,033-761-6911,033-760-4701,37.332496,127.948962,1.739747,2.77,7.26
2,연세대학교원주세브란스기독병원,강원특별자치도 원주시 일산로 20 (일산동),권역응급의료센터,033-741-0114,033-741-1641,37.347955,127.945645,2.116233,2.92,9.51


## **Mission Complete!**

수고 많았습니다!